In [76]:
import pandas as pd
column_names = ['Code', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
min_temps = pd.read_csv('../datasets/raw_data/min_temp.csv', delimiter='\s+', names=column_names)
max_temps = pd.read_csv('../datasets/raw_data/max_temp.csv', delimiter='\s+', names=column_names)
avg_temps = pd.read_csv('../datasets/raw_data/avg_temp.csv', delimiter='\s+', names=column_names)

# Fill the code
min_temps["Code"] = min_temps["Code"].astype(str).str.zfill(10)
max_temps["Code"] = max_temps["Code"].astype(str).str.zfill(10)
avg_temps["Code"] = avg_temps["Code"].astype(str).str.zfill(10)

# Add the type of measure
min_temps["Type"] = "Minimum"
max_temps["Type"] = "Maximum"
avg_temps["Type"] = "Average"

min_temps.head(5)

,Code,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Type
0,0010281895,33.4,26.8,42.4,51.2,58.4,66.5,68.8,69.3,66.4,45.2,41.4,32.7,Minimum
1,0010281896,34.0,36.4,41.2,55.6,63.9,66.9,70.7,70.4,62.6,50.6,45.8,35.1,Minimum
2,0010281897,31.4,41.2,50.4,50.1,55.9,68.3,70.2,68.1,62.5,53.1,41.4,36.9,Minimum
3,0010281898,38.8,34.2,48.5,46.4,60.2,68.0,69.6,69.2,65.1,49.6,38.7,32.5,Minimum
4,0010281899,33.1,28.6,43.5,50.8,63.6,67.6,68.9,69.9,59.4,54.0,42.8,33.8,Minimum


In [77]:
regions = pd.read_csv('../datasets/raw_data/regions_id.csv')
regions["ID"] = regions["ID"].astype(str)
regions["ID"] = regions["ID"].str.zfill(3)
regions.head(5)

,ID,Region
0,001,Alabama
1,002,Arizona
2,003,Arkansas
3,004,California
4,005,Colorado


In [78]:
# Extract the year from the code
min_temps["Year"] = min_temps["Code"].astype(str).str[-4:]
min_temps["Year"] = min_temps["Year"].astype(int)
max_temps["Year"] = max_temps["Code"].astype(str).str[-4:]
max_temps["Year"] = max_temps["Year"].astype(int)
avg_temps["Year"] = avg_temps["Code"].astype(str).str[-4:]
avg_temps["Year"] = avg_temps["Year"].astype(int)

# Extract the region
min_temps["RegionID"] = min_temps["Code"].astype(str).str[:3]
min_temps["Year"] = min_temps["Year"].astype(int)
max_temps["RegionID"] = max_temps["Code"].astype(str).str[:3]
max_temps["Year"] = max_temps["Year"].astype(int)
avg_temps["RegionID"] = avg_temps["Code"].astype(str).str[:3]
avg_temps["Year"] = avg_temps["Year"].astype(int)

# Convert Fahrenheit to Celsius
min_temps[months] = ((min_temps[months] - 32) * 5/9).round(3)
max_temps[months] = ((max_temps[months] - 32) * 5/9).round(3)
avg_temps[months] = ((avg_temps[months] - 32) * 5/9).round(3)


# Merge DataFrames based on 'RegionID' and 'ID'
min_temps = pd.merge(min_temps, regions, left_on='RegionID', right_on='ID', how='left')
max_temps = pd.merge(max_temps, regions, left_on='RegionID', right_on='ID', how='left')
avg_temps = pd.merge(avg_temps, regions, left_on='RegionID', right_on='ID', how='left')


# Drop the redundant 'ID' column and rename 'Region' to 'RegionName'
min_temps = min_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})
max_temps = max_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})
avg_temps = avg_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})


# To set the new order of the columns
new_columns = ["Code", "Type", "RegionID", "RegionName", "Year"] + months
min_temps = min_temps[new_columns]
max_temps = max_temps[new_columns]
avg_temps = avg_temps[new_columns]


min_temps.head(5)

,Code,Type,RegionID,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0010281895,Minimum,001,Alabama,1895,0.778,-2.889,5.778,10.667,14.667,19.167,20.444,20.722,19.111,7.333,5.222,0.389
1,0010281896,Minimum,001,Alabama,1896,1.111,2.444,5.111,13.111,17.722,19.389,21.500,21.333,17.000,10.333,7.667,1.722
2,0010281897,Minimum,001,Alabama,1897,-0.333,5.111,10.222,10.056,13.278,20.167,21.222,20.056,16.944,11.722,5.222,2.722
3,0010281898,Minimum,001,Alabama,1898,3.778,1.222,9.167,8.000,15.667,20.000,20.889,20.667,18.389,9.778,3.722,0.278
4,0010281899,Minimum,001,Alabama,1899,0.611,-1.889,6.389,10.444,17.556,19.778,20.500,21.056,15.222,12.222,6.000,1.000


In [79]:
# Drop null regions
min_temps.dropna(subset=['RegionName'], inplace=True)
max_temps.dropna(subset=['RegionName'], inplace=True)
avg_temps.dropna(subset=['RegionName'], inplace=True)

min_temps['RegionName'] = min_temps['RegionName'].replace('National (contiguous 48 States)', 'National')
max_temps['RegionName'] = max_temps['RegionName'].replace('National (contiguous 48 States)', 'National')
avg_temps['RegionName'] = avg_temps['RegionName'].replace('National (contiguous 48 States)', 'National')

In [80]:
# Union of dataframes
temps = pd.concat([min_temps, max_temps, avg_temps])
temps.head(10)

,Code,Type,RegionID,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0010281895,Minimum,001,Alabama,1895,0.778,-2.889,5.778,10.667,14.667,19.167,20.444,20.722,19.111,7.333,5.222,0.389
1,0010281896,Minimum,001,Alabama,1896,1.111,2.444,5.111,13.111,17.722,19.389,21.500,21.333,17.000,10.333,7.667,1.722
2,0010281897,Minimum,001,Alabama,1897,-0.333,5.111,10.222,10.056,13.278,20.167,21.222,20.056,16.944,11.722,5.222,2.722
3,0010281898,Minimum,001,Alabama,1898,3.778,1.222,9.167,8.000,15.667,20.000,20.889,20.667,18.389,9.778,3.722,0.278
4,0010281899,Minimum,001,Alabama,1899,0.611,-1.889,6.389,10.444,17.556,19.778,20.500,21.056,15.222,12.222,6.000,1.000
5,0010281900,Minimum,001,Alabama,1900,-0.056,0.222,5.167,10.889,14.667,19.667,20.611,20.500,18.889,14.556,6.111,2.056
6,0010281901,Minimum,001,Alabama,1901,1.389,0.167,5.056,7.889,13.944,19.167,21.000,19.889,16.111,9.333,1.500,-0.833
7,0010281902,Minimum,001,Alabama,1902,-0.611,-0.778,6.667,9.611,17.278,19.889,21.167,21.111,16.944,10.333,7.722,1.611
8,0010281903,Minimum,001,Alabama,1903,0.722,2.667,10.278,9.000,14.833,16.778,20.444,20.778,15.444,9.667,3.722,-2.611
9,0010281904,Minimum,001,Alabama,1904,-1.278,3.111,7.667,8.500,13.444,18.278,19.222,19.778,17.611,9.556,4.000,1.556


In [81]:
years_fallback = [2022, 2018, 2013, 2008, 2003, 1998, 1993, 1988, 1983, 1978]
fallback = temps[temps["Year"].isin(years_fallback)]
fallback

,Code,Type,RegionID,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
83,0010281978,Minimum,001,Alabama,1978,-2.889,-2.611,3.778,9.833,14.667,19.222,20.778,20.222,18.444,8.222,7.833,1.111
88,0010281983,Minimum,001,Alabama,1983,0.111,2.667,4.889,7.333,13.722,18.000,20.611,20.667,15.667,11.056,4.667,-0.222
93,0010281988,Minimum,001,Alabama,1988,-0.667,0.333,5.111,9.833,12.056,17.222,20.444,20.833,18.500,8.222,7.000,1.333
98,0010281993,Minimum,001,Alabama,1993,4.222,1.500,4.667,7.722,14.056,18.944,21.389,20.722,16.778,10.444,4.000,1.611
103,0010281998,Minimum,001,Alabama,1998,3.222,3.722,5.611,9.833,16.000,20.556,22.056,20.389,19.167,12.000,7.889,5.556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,1100022003,Average,110,National,2003,0.450,0.439,6.278,11.050,16.100,20.028,24.150,23.700,18.311,13.639,5.750,1.828
7565,1100022008,Average,110,National,2008,-0.961,1.500,5.478,10.222,15.200,20.889,23.378,22.378,18.450,12.211,6.622,-0.072
7570,1100022013,Average,110,National,2013,0.139,1.539,4.950,9.822,16.028,21.328,23.450,22.772,19.422,11.911,5.339,-0.522
7575,1100022018,Average,110,National,2018,0.089,1.950,5.961,9.461,18.622,21.972,24.139,23.061,19.739,12.061,4.450,1.950


In [82]:
radarchart = fallback.copy()
radarchart = radarchart[radarchart['Type'] == 'Average']
radarchart = radarchart[["RegionName","Year", "Jan", "Dec", "Nov", "Oct", "Sep", "Aug", "Jul", "Jun","May","Apr", "Mar", "Feb"]]
radarchart[radarchart["RegionName"]=="Alabama"]

,RegionName,Year,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb
83,Alabama,1978,2.000,8.111,15.111,16.778,24.944,26.278,27.056,25.556,21.111,17.722,10.667,3.778
88,Alabama,1983,5.611,5.444,11.778,17.778,22.222,27.278,27.000,23.611,20.333,14.333,11.111,8.111
93,Alabama,1988,4.833,8.222,13.611,15.111,23.611,26.889,26.056,25.056,20.167,17.111,12.389,7.444
98,Alabama,1993,9.333,7.389,10.944,17.111,23.333,26.722,27.889,25.333,20.444,14.833,10.944,7.889
103,Alabama,1998,8.778,10.778,14.389,19.333,25.222,26.611,27.667,27.056,23.111,16.444,11.889,9.556
108,Alabama,2003,4.444,6.667,14.556,17.500,22.833,26.333,25.833,24.222,22.000,17.389,14.222,8.889
113,Alabama,2008,6.111,9.833,10.556,16.778,23.611,25.778,26.667,25.944,21.278,17.111,12.889,9.556
118,Alabama,2013,10.000,8.833,10.889,18.278,23.889,25.500,25.389,25.556,20.111,16.778,9.889,8.889
123,Alabama,2018,4.667,9.444,10.222,19.667,26.333,26.389,27.056,26.278,23.556,15.278,13.000,14.444
127,Alabama,2022,6.611,9.556,13.000,16.222,23.278,26.278,27.611,26.889,23.056,16.944,14.111,10.000


In [83]:
years_fallback2 = [2022, 2018, 2013, 2008, 2003, 1998, 1993, 1988, 1983, 1978, 1973, 1968]
fallback2 = temps[temps["Year"].isin(years_fallback2)]
fallback2
ridge = fallback2.copy()
ridge = ridge[["Type","RegionName","Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
ridge[ridge["Type"]=="Maximum"].head(5)

,Type,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
73,Maximum,Alabama,1968,11.722,11.056,18.944,24.667,27.389,32.778,32.167,32.889,29.611,25.056,17.111,12.389
78,Maximum,Alabama,1973,12.000,13.833,21.333,21.778,26.833,31.000,32.444,31.500,30.611,26.444,21.111,14.389
83,Maximum,Alabama,1978,6.889,10.167,17.556,25.556,27.556,31.833,33.389,32.333,31.444,25.333,22.389,15.111
88,Maximum,Alabama,1983,11.167,13.556,17.333,21.333,26.944,29.222,33.333,33.889,28.778,24.500,18.889,11.111
93,Maximum,Alabama,1988,10.333,14.500,19.667,24.333,28.278,32.889,31.722,32.889,28.722,22.000,20.222,15.167


In [84]:
# Save into CSV files
min_temps.to_csv("../datasets/min_temps.csv", index=False)
max_temps.to_csv("../datasets/max_temps.csv", index=False)
avg_temps.to_csv("../datasets/avg_temps.csv", index=False)
temps.to_csv("../datasets/temps.csv", index=False)
fallback.to_csv("../datasets/temps_fallback.csv", index=False)
radarchart.to_csv("../datasets/temps_radarchart.csv", index=False)
ridge.to_csv("../datasets/raw_data/temps_ridge.csv", index=False)